In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
import warnings
warnings.filterwarnings("ignore")

* Когда команда не отвечает, то мы знаем, что не ответили все
* В случае ответа знать его могут не все => рассмотрим данные как Presence-Only 

In [3]:
df = pd.read_csv('bs2_final.csv')

In [4]:
df.head()

,avg_tournament,date,team_id,tournament_key,mask,weights_arr,player_id
0,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,6212
1,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,18332
2,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,18036
3,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,22799
4,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,15456


In [5]:
train = df[df['date'] < '2020']

In [6]:
print(train['mask'].value_counts())
pis = 8648878 / (8648878+6541822) #встречаемость

0    8648878
1    6541822
Name: mask, dtype: int64


#### Init

In [7]:
c = 1e-6
train['target'] = train['mask'].apply(lambda x: c if x == 0 else pis)

In [8]:
train[train['mask'] != 1].head()

,avg_tournament,date,team_id,tournament_key,mask,weights_arr,player_id,target
54,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,6212,0.000001
55,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,18332,0.000001
56,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,18036,0.000001
57,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,22799,0.000001
58,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,15456,0.000001


In [9]:
train[train['mask'] == 1].head()

,avg_tournament,date,team_id,tournament_key,mask,weights_arr,player_id,target
0,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,6212,0.569353
1,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,18332,0.569353
2,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,18036,0.569353
3,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,22799,0.569353
4,0.46633,2019-01-05T19:00:00+03:00,45556,4772,1,0.883117,15456,0.569353


In [10]:
from sklearn.preprocessing import OneHotEncoder
encoder = OneHotEncoder().fit(train['player_id'].to_frame())

In [11]:
from scipy.sparse import csr_matrix, hstack
from scipy.special import logit

In [12]:
X_train = hstack([csr_matrix(train[['avg_tournament', 'weights_arr']].values),
                    encoder.transform(train['player_id'].to_frame())])

y_train = logit(train['target'])

In [13]:
n_p = 8648878
n_u = 6541822
- np.log((n_p + pis * n_u)/(pis * n_u))

-1.200594128337945

In [14]:
from sklearn.linear_model import LinearRegression

Реализация по аналогии с http://www.web.stanford.edu/~hastie/Papers/ward_biometrics.pdf

In [15]:
def e_step(X, model):
    pred = model.predict(X)
    adds = np.log((n_p + pis * n_u)/(pis * n_u))
    return np.exp(pred - adds) / (np.exp(pred - adds) + 1)

In [16]:
def m_step(X_train, y_train):
    lr = LinearRegression().fit(X_train, y_train)
    return lr

In [17]:
%%time
lr1 = m_step(X_train, y_train)

CPU times: user 1min 13s, sys: 4.83 s, total: 1min 18s
Wall time: 40.7 s


In [18]:
train['target1'] = e_step(X_train, lr1)

In [19]:
train_upd = train[train['mask'] == 1][['avg_tournament', 'date', 'team_id', 'tournament_key', 'mask',
       'weights_arr', 'player_id', 'target1']]

train_upd.columns = ['avg_tournament', 'date', 'team_id', 'tournament_key', 'mask', 
                     'weights_arr', 'player_id', 'target']

In [20]:
cols = ['avg_tournament', 'date', 'team_id', 'tournament_key', 'mask', 'weights_arr', 'player_id', 'target']

In [21]:
train = pd.concat([train[train['mask'] == 0][cols], train_upd])

In [22]:
train.head(2)

,avg_tournament,date,team_id,tournament_key,mask,weights_arr,player_id,target
54,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,6212,0.000001
55,0.46633,2019-01-05T19:00:00+03:00,45556,4772,0,0.030303,18332,0.000001


In [25]:
lr1.coef_.shape

(53768,)

In [26]:
df_rates = pd.DataFrame()
df_rates['players'] = pd.Series(encoder.get_feature_names())
df_rates['rates'] = pd.Series(lr1.coef_[2:])
df_rates['players'] = df_rates['players'].str[3:].astype(int)

In [28]:
# df_rates.to_csv('bs3_rates.csv', header=True, index=False)

In [29]:
df_tp = pd.read_csv('bs2_players.csv')
df2 = pd.read_csv('bs1_data.csv')
df_rates = pd.read_csv('bs3_rates.csv')

df_rates.columns = ['player_id', 'rates']
ans = df[df['date'] >= '2020'].merge(df_rates, on='player_id', how='left')
ans_gr = ans[['tournament_key', 'team_id', 'rates']].groupby(['tournament_key', 'team_id']).rates.sum().reset_index()

In [30]:
from scipy import stats

In [31]:
res_fin = []
for key in tqdm(ans.tournament_key.unique()):
    ans_sample = ans_gr[ans_gr['tournament_key'] == key].sort_values(by=['tournament_key', 'rates'], ascending=False)
    ans_sample['position_pred'] = np.arange(len(ans_sample))+1
    ans_sample = ans_sample.merge(df2[['tournament_key', 'team_id', 'position']], on=['tournament_key', 'team_id'])
    res_fin.append(stats.spearmanr(ans_sample['position'], ans_sample['position_pred'])[0])

100%|██████████| 161/161 [00:02<00:00, 65.36it/s]


In [32]:
res_fin = [x for x in res_fin if str(x) != 'nan']
len(res_fin)

160

In [33]:
np.mean(res_fin)

0.7338188019451908

In [32]:
np.mean(res_fin)

0.7338188019451908

In [100]:
y_train = logit(train['target'])

In [35]:
# %%time
# lr1 = m_step(X_train, y_train)
# train['target1'] = e_step(X_train, lr1)

In [34]:
for i in range(0,5):

    y_train = logit(train['target'])

    lr1 = m_step(X_train, y_train)
    train['target1'] = e_step(X_train, lr1)

    train_upd = train[train['mask'] == 1][['avg_tournament', 'date', 'team_id', 'tournament_key', 'mask',
           'weights_arr', 'player_id', 'target1']]

    train_upd.columns = ['avg_tournament', 'date', 'team_id', 'tournament_key', 'mask', 
                         'weights_arr', 'player_id', 'target']

    train = pd.concat([train[train['mask'] == 0][cols], train_upd])

    df_rates = pd.DataFrame()
    df_rates['players'] = pd.Series(encoder.get_feature_names())
    df_rates['rates'] = pd.Series(lr1.coef_[2:])
    df_rates['players'] = df_rates['players'].str[3:].astype(int)

    df_rates.columns = ['player_id', 'rates']
    ans = df[df['date'] >= '2020'].merge(df_rates, on='player_id', how='left')
    ans_gr = ans[['tournament_key', 'team_id', 'rates']].groupby(['tournament_key', 'team_id']).rates.sum().reset_index()

    res_fin = []
    for key in tqdm(ans.tournament_key.unique()):
        ans_sample = ans_gr[ans_gr['tournament_key'] == key].sort_values(by=['tournament_key', 'rates'], ascending=False)
        ans_sample['position_pred'] = np.arange(len(ans_sample))+1
        ans_sample = ans_sample.merge(df2[['tournament_key', 'team_id', 'position']], on=['tournament_key', 'team_id'])
        res_fin.append(stats.spearmanr(ans_sample['position'], ans_sample['position_pred'])[0])

    res_fin = [x for x in res_fin if str(x) != 'nan']
    len(res_fin)

    print(i, np.mean(res_fin))

100%|██████████| 161/161 [00:02<00:00, 66.94it/s]


0 -0.1422783863267874


100%|██████████| 161/161 [00:02<00:00, 60.78it/s]


1 -0.3489536062141627


100%|██████████| 161/161 [00:02<00:00, 62.37it/s]


2 -0.4321961450853754


100%|██████████| 161/161 [00:02<00:00, 62.29it/s]


3 -0.48046076685614725


100%|██████████| 161/161 [00:02<00:00, 60.65it/s]

4 -0.4762338622437112


Есть ощущение, что из-за первичного приближения неверных ответов модель имеет склонность занижать предсказания

### Рейтинги турниров

In [33]:
with open('tournaments.pkl', 'rb') as f:
    tour = pickle.load(f)

In [34]:
tour = pd.DataFrame(tour).T
tour = tour[tour['dateStart'] >= '2020']
tour.columns

Index(['id', 'name', 'dateStart', 'dateEnd', 'type', 'season', 'orgcommittee',
       'synchData', 'questionQty'],
      dtype='object')

In [41]:
top_tour = df.merge(df_rates, on='player_id').groupby(['tournament_key']).rates.sum()\
                        .reset_index().sort_values(by='rates', ascending=False)

In [47]:
# top_tour.head()

In [45]:
#сложные турниры
top_tour.merge(tour[['id', 'name']], left_on = 'tournament_key', right_on = 'id')[['tournament_key', 'name']].head(10)

,tournament_key,name
0,5719,Синхрон Моносова
1,5753,Балтийский Берег. 3 игра
2,6093,Азовский бриз
3,5754,Балтийский Берег. 4 игра
4,5823,ОВСЧ. 6 этап
5,5923,"Фрегат ""Паллада"": синхрон ""Борского корабела"""
6,6041,Кубок Бесконечности. V этап
7,6042,Кубок Бесконечности. VI этап
8,6142,Молодёжный кубок Европы. Этап 1
9,6070,Синхронный Кубок Хайфы


In [46]:
#простые турниры
top_tour.merge(tour[['id', 'name']], left_on = 'tournament_key', right_on = 'id')[['tournament_key', 'name']].tail(10)

,tournament_key,name
151,6182,"Кубок Ростова-на-Дону, день 3"
152,6314,Студенческий чемпионат Ярославской области
153,6362,Онлайн: Delivery Cup
154,6340,Покорение Сибири. День 2
155,6409,Онлайн: 08:00 Лига старта-13
156,6256,50 на двоих
157,6181,"Кубок Ростова-на-Дону, день 2"
158,6223,С чаем пойдёт (green)
159,6339,Покорение Сибири. День 1
160,6325,Буслик
